<a href="https://colab.research.google.com/github/H3IN-THANT/ShadowFox/blob/main/ImageTagging_Beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import os

In [ ]:

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 1
NUM_CLASSES = 10

In [ ]:
def loadAndPreData():

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

  VALIDATION_SPLIT = 0.15
  test_imgs = x_test.shape[0]
  num_val_samples = int(VALIDATION_SPLIT * test_imgs)

  x_val, x_test = x_test[:num_val_samples], x_test[num_val_samples:]
  y_val, y_test = y_test[:num_val_samples], y_test[num_val_samples:]

  def make_ds(x, y):
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(lambda x, y: (tf.image.resize(x, [IMG_HEIGHT, IMG_WIDTH])/255.0, y))
    return ds.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

  # def preprocess_img(image, label):
  #   image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
  #   image = tf.cast(image, tf.float32)/255.0
  #   return image, label

  # train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  # val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val))
  # test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))

  # train_ds = train_ds.map(preprocess_img).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
  # val_ds = val_ds.map(preprocess_img).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
  # test_ds = test_ds.map(preprocess_img).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

  x_train = x_train.astype('float32') / 255.0
  x_test = x_test.astype('float32') / 255.0
  x_val = x_val.astype('float32') / 255.0

  y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
  y_val = tf.keras.utils.to_categorical(y_val, NUM_CLASSES)
  y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

  train_ds = make_ds(x_train, y_train)
  val_ds = make_ds(x_val, y_val)
  test_ds = make_ds(x_test, y_test)

  return (x_train, y_train), (x_val, y_val), (x_test, y_test), train_ds, val_ds, test_ds

In [ ]:
# def build_CNN():
#     model = models.Sequential([
#         layers.InputLayer(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

#         layers.Conv2D(32, (3,3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.Conv2D(32, (3,3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D((2,2)),

#         layers.Conv2D(64, (3,3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D((2,2)),

#         layers.Conv2D(128, (3,3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D((2,2)),

#         layers.Conv2D(256, (3,3), activation='relu', padding='same'),
#         layers.BatchNormalization(),
#         layers.MaxPooling2D((2,2)),

#         layers.Flatten(),
#         layers.Dense(256, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(NUM_CLASSES, activation='softmax')
#     ])
#     return model

In [ ]:
def build_CNN():
    model = models.Sequential([
        layers.InputLayer(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.2),

        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.3),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.4),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.5),

        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.6),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

In [ ]:
def create_dataGen():
  datagen = ImageDataGenerator(
      rotation_range = 30,
      width_shift_range = 0.15,
      height_shift_range = 0.15,
      shear_range = 0.15,
      zoom_range = 0.15,
      horizontal_flip = True,
      fill_mode = 'nearest'
  )

  return datagen

In [ ]:
def get_optimizer(learning_rate= 0.001):
  return tf.keras.optimizers.Adam(learning_rate = learning_rate)

In [ ]:
def eval_model(model, test_ds):
  test_loss, test_acc = model.evaluate(test_ds, verbose = 2)
  print('\nTest Accuracy: {:.2f}%'.format(test_acc*100))

  print("Generating classification report...")
  y_pred_probs = model.predict(test_ds)
  y_pred = np.argmax(y_pred_probs, axis = 1)

  y_true = np.concatenate([np.argmax(y, axis=1) for _, y in test_ds], axis=0)

  report = classification_report(y_true, y_pred) # Remove targetnames as it requires directory access
  print(report)

  print("Confusion matrix...")
  cm = confusion_matrix(y_true, y_pred)
  print(cm)

In [ ]:
def main():
  print("Loading and preprocessing data...")
  (x_train, y_train), (x_val, y_val), (x_test, y_test), train_ds, val_ds, test_ds = loadAndPreData()

  print("Creating data generator with augmentation...")
  datagen = create_dataGen()
  # datagen.fit(x_train) # No longer needed as we are using tf.data.Dataset

  print("Building CNN model...")
  model = build_CNN()
  model.summary()

  print("Model input shape:", model.input_shape)
  print("Model output shape:", model.output_shape)

  optimizer = get_optimizer(learning_rate = 0.0005)
  model.compile(optimizer = optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

  print("Training model with aug_data...")
  history = model.fit(train_ds,
                      epochs = EPOCHS, validation_data= val_ds, verbose = 1)

  print("Evaluating model on test data...")
  eval_model(model, test_ds)

  save_path = "image_tagging_model.h5"
  model.save(save_path)
  print(f"Model saved to {save_path}")

  print("Loading saved model for test prediction...")
  loaded_model = tf.keras.models.load_model(save_path)

  sample_img = x_test[0:1]
  sample_img = tf.image.resize(sample_img, [IMG_HEIGHT, IMG_WIDTH])
  sample_img = sample_img.numpy().astype("float32") /255.0

  sample_pred = loaded_model.predict(sample_img)
  print(sample_pred)

  class_idx = np.argmax(sample_pred, axis = 1)[0]
  class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

  predicted_class = class_names[class_idx]

  print(f"Predicted class for the sample image: {predicted_class}")

In [ ]:
if __name__ == "__main__":
  main()

Loading and preprocessing data...
Creating data generator with augmentation...
Building CNN model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_23          │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 14,025,002 (53.50 MB)

 Trainable params: 14,022,570 (53.49 MB)

 Non-trainable params: 2,432 (9.50 KB)

Model input shape: (None, 224, 224, 3)
Model output shape: (None, 10)
Training model with aug_data...
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 272s 164ms/step - accuracy: 0.3294 - loss: 2.1305 - val_accuracy: 0.5573 - val_loss: 1.2759
Evaluating model on test data...
266/266 - 12s - 44ms/step - accuracy: 0.5465 - loss: 1.2717

Test Accuracy: 54.65%
Generating classification report...
266/266 ━━━━━━━━━━━━━━━━━━━━ 13s 46ms/step


              precision    recall  f1-score   support

           0       0.63      0.51      0.56       851
           1       0.76      0.65      0.70       848
           2       0.37      0.54      0.44       854
           3       0.38      0.41      0.39       849
           4       0.52      0.46      0.49       863
           5       0.43      0.55      0.48       874
           6       0.75      0.53      0.62       832
           7       0.81      0.46      0.59       851
           8       0.50      0.82      0.62       835
           9       0.76      0.54      0.63       843

    accuracy                           0.55      8500
   macro avg       0.59      0.55      0.55      8500
weighted avg       0.59      0.55      0.55      8500

Confusion matrix...
[[436  15 110  12   3  12   7   6 233  17]
 [ 27 553  25   7   3   9   9   6 126  83]
 [ 50   6 464  59  86 102  26  10  49   2]
 [ 24  12 141 345  51 193  27  12  40   4]
 [ 26   1 191  84 401  72  39  21  27   1]
 [ 16 

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 719ms/step
[[6.9655548e-04 9.5020086e-01 2.1980217e-05 1.9028497e-05 5.7228168e-05
  6.3901321e-06 7.3932017e-05 3.2292810e-05 4.1555660e-04 4.8476208e-02]]
Predicted class for the sample image: automobile
